Order

In [18]:
import heapq
class Order:
    def __init__(self, order_id, side, price, qty, timestamp):
        self.order_id = order_id
        self.side = side  # 'buy' or 'sell'
        self.price = price
        self.qty = qty
        self.timestamp = timestamp
    def __lt__(self, other):
        if self.price == other.price:
            return self.timestamp < other.timestamp
        else :
            return self.price < other.price

Orderbook

In [21]:
class OrderBook:
    def __init__(self):
        self.bids = []
        self.asks = []
        self.trades = []
    
    def add_order(self, order):
        if order.side == 'buy':
            self.match_order(order, self.asks, is_buy=True)
        else:
            self.match_order(order, self.bids, is_buy=False)
    
    def match_order(self, incoming, opposite_heap, is_buy):
        while incoming.qty > 0 and opposite_heap:
            best_price_initial , best_order = opposite_heap[0]
            best_price = -best_price_initial if not is_buy else best_price_initial
            if (is_buy and incoming.price >= best_price) or (not is_buy and incoming.price <= best_price):
                trade_qty = min(incoming.qty, best_order.qty)
                # Execute Trade
                print(f"TRADE: {trade_qty} @ {best_price} (IDs: {incoming.order_id} & {best_order.order_id})")

                self.trades.append((incoming.order_id, best_order.order_id, trade_qty, best_price))
                incoming.qty -= trade_qty 
                best_order.qty -= trade_qty

                if best_order.qty ==0:
                    heapq.heappop(opposite_heap) 
            else:
                break
        if incoming.qty > 0:
            if is_buy:
                heapq.heappush(self.bids, (-incoming.price, incoming))
            else:
                heapq.heappush(self.asks, (incoming.price, incoming))
    
    def print_depth(self, levels=5):
        print("\n--- Market Depth ---")
        top_asks = sorted(self.asks[:levels])
        for p, o in reversed(top_asks):
            print(f"ASK: {p} | Qty: {o.qty}")
        
        print("----------")
        
        top_bids = sorted(self.bids[:levels])
        for neg_p, o in top_bids:
            print(f"BID: {-neg_p} | Qty: {o.qty}")
        
            

Dataset

In [25]:
ob = OrderBook()

# Sample Orders (Timestamp is just an integer count)
orders = [
    Order(1, 'sell', 105, 10, 1),
    Order(2, 'sell', 103, 5, 2),
    Order(3, 'buy', 101, 20, 3),
    Order(4, 'buy', 104, 15, 4), # This will match with ID 2 (Price 103)
    Order(5, 'buy', 107, 8, 5),
    Order(6, 'sell', 103, 15, 2)
]

for o in orders:
    print(f"Adding Order {o.order_id}: {o.side} {o.qty} @ {o.price}")
    ob.add_order(o)
    ob.print_depth(levels=5)

Adding Order 1: sell 10 @ 105

--- Market Depth ---
ASK: 105 | Qty: 10
----------
Adding Order 2: sell 5 @ 103

--- Market Depth ---
ASK: 105 | Qty: 10
ASK: 103 | Qty: 5
----------
Adding Order 3: buy 20 @ 101

--- Market Depth ---
ASK: 105 | Qty: 10
ASK: 103 | Qty: 5
----------
BID: 101 | Qty: 20
Adding Order 4: buy 15 @ 104
TRADE: 5 @ 103 (IDs: 4 & 2)

--- Market Depth ---
ASK: 105 | Qty: 10
----------
BID: 104 | Qty: 10
BID: 101 | Qty: 20
Adding Order 5: buy 8 @ 107
TRADE: 8 @ 105 (IDs: 5 & 1)

--- Market Depth ---
ASK: 105 | Qty: 2
----------
BID: 104 | Qty: 10
BID: 101 | Qty: 20
Adding Order 6: sell 15 @ 103
TRADE: 10 @ 104 (IDs: 6 & 4)

--- Market Depth ---
ASK: 105 | Qty: 2
ASK: 103 | Qty: 5
----------
BID: 101 | Qty: 20


Deterministic Verification

In [26]:
# Create two fresh books
book1 = OrderBook()
book2 = OrderBook()

# Process the same list through both
for o in orders: book1.add_order(o)
for o in orders: book2.add_order(o)

# The ultimate test
assert book1.trades == book2.trades, "The engine is NOT deterministic!"
print("Verification Complete: Engine is deterministic.")

Verification Complete: Engine is deterministic.
